## Loading track2p results 

In [ ]:
# imports
import os
from types import SimpleNamespace
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.colors as mcolors
import numpy as np

In [ ]:
t2p_save_path = '/Volumes/data_jm_share/data_proc/jm/jm046/chan1'  # (change this based on your data)
save_path='/Volumes/data_jm_share/data_proc/jm/jm046'
plane = 'plane0' # which plane to process (the example dataset is single-plane)

In [ ]:
# np.load() the match matrix (plane0_match_mat.npy)
t2p_match_mat = np.load(os.path.join(t2p_save_path,'track2p', f'{plane}_match_mat.npy'), allow_pickle=True)

# np.load() settings (this contains suite2p paths etc.) (track_ops.npy)
track_ops_dict = np.load(os.path.join(t2p_save_path, 'track2p', 'track_ops.npy'), allow_pickle=True).item()
track_ops = SimpleNamespace(**track_ops_dict) # create dummy object from the track_ops dictionary

print(t2p_save_path)
print(track_ops.save_path)
print(track_ops.reg_chan)
print(track_ops.transform_type)
print(t2p_match_mat.shape)
print(t2p_match_mat)

In [ ]:
t2p_match_mat_notnone = t2p_match_mat != None #bolean matrix
print(t2p_match_mat_notnone)
plt.figure(figsize=(20, 1), dpi=500)
plt.imshow(t2p_match_mat_notnone.T, aspect='auto', vmin=0, vmax=1)


In [ ]:
print('Datasets used for t2p:\n')
for ds_path in track_ops.all_ds_path:
    iscell=np.load(os.path.join(ds_path,'suite2p', 'plane0','iscell.npy'))
    iscell = iscell[iscell[:, 0] > track_ops.iscell_thr]
    print(len(iscell))


## Above is the output of the algorithm, now generate ground truth
### Generate grid

In [ ]:
# load the suite2p output of the first dataset
s2p_path = os.path.join(track_ops.all_ds_path[0], 'suite2p', plane)
print(track_ops.all_ds_path)

# load the ops and stat
ops_first = np.load(os.path.join(s2p_path, 'ops.npy'), allow_pickle=True).item()
stat_first = np.load(os.path.join(s2p_path, 'stat.npy'), allow_pickle=True)
iscell_first = np.load(os.path.join(s2p_path, 'iscell.npy'), allow_pickle=True)
s2p_inds = np.arange(len(stat_first))

# filter stat based on the track2p probability
print(f'Filtering stat based on the track2p probability (iscell_thr={track_ops.iscell_thr})')
s2p_inds_iscell = s2p_inds[iscell_first[:, 0] > track_ops.iscell_thr]
stat_first_iscell = stat_first[iscell_first[:, 0] > track_ops.iscell_thr]
print(f'Filtered stat from {len(stat_first)} to {len(stat_first_iscell)} cells')



In [ ]:
# now plot the fov with contours
plt.figure(figsize=(10, 10))
plt.imshow(ops_first['meanImg'], cmap='gray')
for i in range(len(stat_first_iscell)):
    # get random color 
    color = np.random.rand(3)
    plt.scatter(stat_first_iscell[i]['xpix'], stat_first_iscell[i]['ypix'], s=1, color=color, alpha=0.5)

In [ ]:
# now make a 8x8 grid on the image
n_grid = 9
grid_x = np.arange(0, ops_first['meanImg'].shape[1], ops_first['meanImg'].shape[1] // n_grid)
grid_y = np.arange(0, ops_first['meanImg'].shape[0], ops_first['meanImg'].shape[0] // n_grid)

plt.imshow(ops_first['meanImg'], cmap='gray')
for i in range(n_grid):
    plt.axvline(grid_x[i], color='r')
    plt.axhline(grid_y[i], color='r')

plt.axis('off')

In [ ]:
# now for each grid cross get the nearest cell (except for 0 and 512 pixels)
all_cell_med = []

# get all cell medians from stat
for i in range(len(stat_first_iscell)):
    all_cell_med.append(stat_first_iscell[i]['med'])
    

In [ ]:
# compute coordinates of the grid crosses
grid_crosses = []
for i in range(1, n_grid):
    for j in range(1, n_grid):
        grid_crosses.append((grid_x[i], grid_y[j]))

In [ ]:
grid_crosses

In [ ]:
plt.imshow(ops_first['meanImg'], cmap='gray')
for i in range(n_grid):
    plt.axvline(grid_x[i], color='gray')
    plt.axhline(grid_y[i], color='gray')
for cross in grid_crosses:
    plt.scatter(cross[0], cross[1], s=20, color='r')

plt.axis('off')


In [ ]:
# compute distances between each cell and each grid cross
distances = np.zeros((len(stat_first_iscell), len(grid_crosses)))
for i, cell_med in enumerate(all_cell_med):
    for j, cross in enumerate(grid_crosses):
        distances[i, j] = np.sqrt((cell_med[0] - cross[0])**2 + (cell_med[1] - cross[1])**2)

In [ ]:
# now get the nearest cell for each grid cross
nearest_cells = np.argmin(distances, axis=0)

In [ ]:
nearest_cells

In [ ]:
plt.imshow(ops_first['meanImg'], cmap='gray')

for cross in grid_crosses:
    plt.scatter(cross[0], cross[1], s=20, color='C0')

plt.axis('off')

for i, cell in enumerate(nearest_cells):
    roi = np.zeros_like(ops_first['meanImg'])
    roi[stat_first_iscell[cell]['ypix'], stat_first_iscell[cell]['xpix']] = 1
    plt.contour(roi, levels=[0.5], colors='C1')

In [ ]:
# now get the original indices
original_indices = s2p_inds_iscell[nearest_cells]
print('Now open suite2p and try to manually track cells:')
[print(f'{original_indices[i]}') for i in range(len(original_indices))]

In [ ]:
# make the same plot as above but with numbers labeled next to the roi
plt.figure(figsize=(10, 10))
# clip based on 99.99 percentile
img = ops_first['meanImg'].copy()
img[img > np.percentile(img, 99.99)] = np.percentile(img, 99.99)
plt.imshow(img, cmap='gray')

for i, cell in enumerate(nearest_cells):
    roi = np.zeros_like(ops_first['meanImg'])
    roi[stat_first_iscell[cell]['ypix'], stat_first_iscell[cell]['xpix']] = 1
    plt.contour(roi, levels=[0.5], colors='C1')
    plt.text(stat_first_iscell[cell]['med'][1]+10, stat_first_iscell[cell]['med'][0]-10, f'{original_indices[i]}', color='C1')
    plt.scatter(grid_crosses[i][0], grid_crosses[i][1], s=20, color='C0', marker='x')

plt.axis('off')

In [ ]:
np.sort(original_indices)

### Getting real suite2p indexes to compare it to

In [ ]:
def function(path, plane):

    track_ops_dict = np.load(os.path.join(path, "track2p", "track_ops.npy"), allow_pickle=True).item()
    track_ops = SimpleNamespace(**track_ops_dict)
    t2p_match_mat = np.load(os.path.join(path,"track2p" ,f"{plane}_match_mat.npy"), allow_pickle=True)
    #t2p_match_mat_allday = t2p_match_mat
    
    if track_ops.iscell_thr is not None:
        true_indices= np.empty((len(t2p_match_mat), len(track_ops.all_ds_path)), dtype=object)
        
    for j in range(len(t2p_match_mat)):
        for (i, ds_path) in enumerate(track_ops.all_ds_path):
            iscell = np.load(os.path.join(ds_path, 'suite2p', plane, 'iscell.npy'), allow_pickle=True)
            if track_ops.iscell_thr is not None:
                s2p_indexes= np.where(iscell[:,1]>track_ops.iscell_thr)[0]
                t2p_index=t2p_match_mat[j,i]

                if t2p_index is None:
                    true_index = None
                else:
                    true_index=s2p_indexes[t2p_index]

                true_indices[j, i] = true_index
            else:
                pass
    if track_ops.iscell_thr is not None:        
        np.save(os.path.join(path, "true_indices.npy"), true_indices)
        print("saved")
    else:
        print("not in locals")
        
function(t2p_save_path, plane)  

indices_npy= np.load(os.path.join(t2p_save_path,"true_indices.npy"), allow_pickle=True)
print(indices_npy)
print(indices_npy.shape)

#indices_npy (index of the cell in the suite2p output)

In [ ]:
# index rows for cells where the first row is in original_indices
all_s2p_idxs = []
for idx in np.sort(original_indices):
    idx_row = np.where(indices_npy[:,0]==idx)
    print(idx_row)

    if len(idx_row[0]) == 0:
        s2p_idxs = [None]*len(track_ops.all_ds_path)
        #print('here')

    else:
        s2p_idxs = indices_npy[idx_row]
    all_s2p_idxs.append(s2p_idxs[0])

all_s2p_idxs = np.array(all_s2p_idxs)

#print(all_s2p_idxs.dtype)
print(f'all_s2p_idxs {all_s2p_idxs}')

### Loading and processing cellreg outputs

In [ ]:
# Create .mat files for cellreg from suite2p outputs

import numpy as np
import os
from scipy.io import savemat
import os
import numpy as np
import matplotlib.colors as mcolors
import random
from types import SimpleNamespace

for (i, ds_path) in enumerate(track_ops.all_ds_path):
    stat = np.load(os.path.join(ds_path, 'suite2p', f'plane{plane}' ,'stat.npy'), allow_pickle=True)
    iscell = np.load(os.path.join(ds_path, 'suite2p', f'plane{plane}', 'iscell.npy'), allow_pickle=True)
    ops = np.load(os.path.join(ds_path, 'suite2p', f'plane{plane}', 'ops.npy'), allow_pickle=True).item()


    Ly, Lx = int(ops['Ly']), int(ops['Lx'])


    N = int(np.sum(iscell[:, 0]))

    footprints = np.zeros((N, Ly, Lx))


    neuron_idx = 0
    for i, s in enumerate(stat):
        if iscell[i, 0]: 
            footprints[neuron_idx, s['ypix'], s['xpix']] = s['lam']
            neuron_idx += 1


    print(f"Footprints shape: {footprints.shape}")

    footprints_part1 = footprints[:N//2]
    footprints_part2 = footprints[N//2:]


    savemat(os.path.join(ds_path,"footprints_part1.mat"), 
        {'footprints_part1': footprints_part1})

    savemat(os.path.join(ds_path,"footprints_part2.Mat"), 
        {'footprints_part2': footprints_part2})


In [ ]:
import h5py

# this is the directory that contains a /track2p folder that is output by running the track2p algorithm
cellreg_path = os.path.join(save_path, 'cellreg')
print(cellreg_path)

with h5py.File(os.path.join(cellreg_path, "cellRegistered.mat"), 'r') as f:
    # Accéder au dataset spécifique
    cell_to_index_map = f['cell_registered_struct/cell_to_index_map']
 
    cell_to_index_map = np.array(cell_to_index_map)
    print(cell_to_index_map.shape)

    # Filtrer les colonnes où la première ligne n'est pas égale à 0
    # Pour avoir la même shape que le tableau de match de track2p (autant de lignes que dans iscell > 0.5 du jour 1)
    mask0 = cell_to_index_map[0, :] != 0
    filtered_cell_to_index_map = cell_to_index_map[:, mask0]
    print(filtered_cell_to_index_map.astype(int))

    # Convertir en entier
    filtered_cell_to_index_map = filtered_cell_to_index_map.astype(int)

    # Remplacer les zéros par None et décrémenter les autres chiffres de 1
    filtered_cell_to_index_map = np.where(filtered_cell_to_index_map == 0, None, filtered_cell_to_index_map - 1)
    print(filtered_cell_to_index_map)

    # Enregistrer le tableau modifié
    np.save(os.path.join(cellreg_path, "plane0_match_mat_cellreg.npy"), filtered_cell_to_index_map)
    print("NumPy file 'plane0_match_mat_cellreg.npy' created successfully.")

    # Charger et afficher le fichier NumPy modifié
    cellreg_match_mat = np.load(os.path.join(cellreg_path, "plane0_match_mat_cellreg.npy"), allow_pickle=True)
    print(cellreg_match_mat)
    print(cellreg_match_mat.shape)

In [ ]:
# Afficher le tableau modifié avec plt.imshow
bool_array = np.where(cellreg_match_mat == None, 0, 1).astype(bool)
plt.figure(figsize=(20, 1), dpi=500)
plt.imshow(bool_array,  aspect='auto', vmin=0, vmax=1)
plt.show()

In [ ]:
cellreg_match_mat = cellreg_match_mat.T

In [ ]:
cellreg_match_mat

In [ ]:
for i in range(len(cellreg_match_mat)):
    if all(x is None for x in cellreg_match_mat[i][1:]):
        cellreg_match_mat[i] = None

In [ ]:
cellreg_match_mat

In [ ]:
# For same as t2p_match_mat
for i in range(len(cellreg_match_mat)):
    if cellreg_match_mat[i,1] is None:
        cellreg_match_mat[i] = None

#ne pas faire si cellreg pairs

#### Getting real suite2p indexes to compare it to

In [ ]:
def function_cellreg(path, plane):

    if track_ops.iscell_thr is not None:
        true_indices= np.empty(cellreg_match_mat.shape, dtype=object)
        print(true_indices.shape)
        
    for (i, ds_path) in enumerate(track_ops.all_ds_path):
        iscell = np.load(os.path.join(ds_path, 'suite2p', plane, 'iscell.npy'), allow_pickle=True)
        s2p_indexes= np.where(iscell[:,1]>track_ops.iscell_thr)[0]
        #print(s2p_indexes)
        print(f'suite2P {len(s2p_indexes)}')
        for j in range(cellreg_match_mat.shape[0]):
                if track_ops.iscell_thr is not None:
                    cellreg_index=cellreg_match_mat[j,i]
                    print(cellreg_index)

                if cellreg_index is None :
                    true_index = None
                else:
                    true_index=s2p_indexes[cellreg_index]

                true_indices[j, i] = true_index
        else:
            pass
    if track_ops.iscell_thr is not None:        
        np.save(os.path.join(path, "true_indices_cellreg.npy"), true_indices)
        print("saved")
    else:
        print("not in locals")
        
function_cellreg(cellreg_path, plane)  

indices_cellreg_npy= np.load(os.path.join(cellreg_path,"true_indices_cellreg.npy"), allow_pickle=True)
print(indices_cellreg_npy)
print(indices_cellreg_npy.shape)


In [ ]:
# index rows for cells where the first row is in original_indices
all_s2p_idxs_cellreg = []
for idx in np.sort(original_indices):
    idx_row = np.where(indices_cellreg_npy[:,0]==idx)
    print(idx_row)

    if len(idx_row[0]) == 0:
        s2p_idxs = [None]*len(track_ops.all_ds_path)
        #print('here')

    else:
        s2p_idxs = indices_cellreg_npy[idx_row]
    all_s2p_idxs_cellreg.append(s2p_idxs[0])

all_s2p_idxs_cellreg = np.array(all_s2p_idxs_cellreg)

#print(all_s2p_idxs.dtype)
print(all_s2p_idxs_cellreg)

### Loading and processing 'ground truth' (csv table)

In [ ]:
# load the data
print(save_path)
table = np.genfromtxt(os.path.join(save_path, 'ground_truth.csv'), delimiter=';', skip_header=1)
sorted_indices = np.argsort(table[:, 0])
table = table[sorted_indices]
table = np.where(np.isnan(table), None, table)
table = [[int(x) if x is not None else None for x in row] for row in table]
table

In [ ]:
for i in range(len(table)):
    if all(x is None for x in table[i][1:]):
        table[i] = None


In [ ]:
for i in range(len(table)):
    # print the rows
    print(f'track2p:     {all_s2p_idxs[i]}')
    print(f'manual:      {table[i]}')
    print('')

# Compute metrics

## For 'all days" registration

###  For 'all days' evaluation

In [ ]:
def compute_metrics_all_days(all_s2p_idxs, table, path):

    all_s2p_idxs_copy = all_s2p_idxs.copy()
    table_copy = table.copy()

# Because we want only cell present on all days 
    for i in range(len(table)):
        if all_s2p_idxs_copy[i] is not None:
            if any(x is None for x in all_s2p_idxs_copy[i]):
                all_s2p_idxs_copy[i] = None
        if table_copy[i] is not None:
            if any(x is None for x in table_copy[i]):
                table_copy[i] = None

        print(f'track2p:     {all_s2p_idxs_copy[i]}')
        print(f'manual:      {table_copy[i]}')


    FP_nb=0
    TP_nb=0
    FN_nb=0
    IS_nb=0
    TN_nb=0

    for i in range(len(table_copy)):
        if all_s2p_idxs_copy[i] is None and table_copy[i] is None:
                TN_nb +=1
        # if track2p detected on all days 
        elif all_s2p_idxs_copy[i] is None and table_copy[i] is not None:
                FN_nb +=1
        elif all_s2p_idxs_copy[i] is not None and table_copy[i] is None:
                FP_nb +=1
        elif all_s2p_idxs_copy[i] is not None and table_copy[i] is not None:
            if any((x!=y) for x, y in zip(all_s2p_idxs_copy[i], table_copy[i])):
                    IS_nb += 1
            else:
                    TP_nb +=1

    TP_rate= TP_nb/len(table)
    TN_rate= TN_nb/len(table)
    FP_rate= FP_nb/len(table)
    FN_rate= FN_nb/len(table)
    IS_rate= IS_nb/len(table)

    print(f'Sum of rates: {TP_rate + TN_rate + FP_rate + FN_rate + IS_rate:.2f}')


 # Calcul de la précision
    if (TP_nb + FP_nb + IS_nb) != 0:
        precision = TP_nb / (TP_nb + FP_nb + IS_nb)
    else:
        precision = 0

    # Calcul du rappel
    if (TP_nb + FN_nb) != 0:
        recall = TP_nb / (TP_nb + FN_nb)
    else:
        recall = 0

    # Calcul du F1-score
    if (precision + recall) != 0:
        f1 = (2 * precision * recall) / (precision + recall)
    else:
        f1 = 0

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1: {f1}')

    metrics = np.array([
    ['TP', TP_rate],
    ['TN', TN_rate],
    ['FP', FP_rate],
    ['FN', FN_rate],
    ['IS', IS_rate],
    ['Precision', round(precision, 2)],
    ['Recall', round(recall, 2)],
    ['F1', round(f1, 2)]
    ], dtype=object)


    filename = os.path.join(path, 'metrics_t2p_all_days.npy')
    np.save(filename, metrics)
    print(f"NumPy file '{filename}' created successfully.")

In [ ]:
compute_metrics_all_days(all_s2p_idxs, table, t2p_save_path)

In [ ]:
metrics_t2p_all_days = np.load(os.path.join(t2p_save_path, 'metrics_t2p_all_days.npy'), allow_pickle=True)
print(metrics_t2p_all_days)

### For 'pairwise' evaluation

In [ ]:
def calculate_metrics_pairs(manual_matches_elements,all_s2p_idxs_elements):
    
    FP_nb=0
    TP_nb=0
    FN_nb=0
    IS_nb=0
    TN_nb=0


    for i in range(len(manual_matches_elements)):
        if all_s2p_idxs_elements[i][1] is None  and manual_matches_elements[i][1] is None:
                TN_nb +=1

        
        # if track2p detected on all days 
        elif all_s2p_idxs_elements[i][1] is None and manual_matches_elements[i][1] is not None:
                FN_nb +=1
      
      
        elif all_s2p_idxs_elements[i][1] is not  None and manual_matches_elements[i][1] is None:
                FP_nb +=1
    
        elif all_s2p_idxs_elements[i][1] is not None and manual_matches_elements[i][1] is not None:
            if all_s2p_idxs_elements[i][1] != manual_matches_elements[i][1]:
                    IS_nb += 1
            else:
                    TP_nb +=1
                    print (manual_matches_elements[i][1])
                    print (all_s2p_idxs_elements[i][1])
               


    TP_rate= TP_nb/len(table)
    TN_rate= TN_nb/len(table)
    FP_rate= FP_nb/len(table)
    FN_rate= FN_nb/len(table)
    IS_rate= IS_nb/len(table)

    print(f'TN: {TN_nb}')
    print(f'FN: {FN_nb}')

    print(f'Sum of rates: {TP_rate + TN_rate + FP_rate + FN_rate + IS_rate:.2f}')


 # Calcul de la précision
    if (TP_nb + FP_nb + IS_nb) != 0:
        precision = TP_nb / (TP_nb + FP_nb + IS_nb)
    else:
        precision = 0

    # Calcul du rappel
    if (TP_nb + FN_nb) != 0:
        recall = TP_nb / (TP_nb + FN_nb)
    else:
        recall = 0

    # Calcul du F1-score
    if (precision + recall) != 0:
        f1 = (2 * precision * recall) / (precision + recall)
    else:
        f1 = 0

    return [TP_rate, TN_rate, FP_rate, FN_rate, IS_rate, precision, recall, f1]

  


#### track2p

In [ ]:
metrics_names = ['TP', 'TN', 'FP', 'FN', 'IS', 'Precision', 'Recall', 'F1']
nb_days = 7

results = np.empty((len(metrics_names), nb_days), dtype=object)
results[:, 0] = metrics_names

print(results)

In [ ]:
for day in range(1 , t2p_match_mat.shape[1]): 

    manual_matches_elements = [(sublist[0], sublist[day]) if sublist is not None and len(sublist) > day else (None, None) for sublist in table]
    all_s2p_idxs_elements = [(sublist[0], sublist[day]) if sublist is not None and len(sublist) > day else (None, None) for sublist in all_s2p_idxs]

    print("manual_matches_elements:", manual_matches_elements)
    print("all_s2p_idxs_elements:", all_s2p_idxs_elements)

    metrics = calculate_metrics_pairs(manual_matches_elements, all_s2p_idxs_elements)
    results[:, day] = metrics

np.save(os.path.join(t2p_save_path , 'metrics_table_pairs.npy'), results)



In [ ]:
results_npy = np.load(os.path.join(t2p_save_path, 'metrics_table_pairs.npy'), allow_pickle=True)
print(results_npy)
print(t2p_save_path)

#### cellreg

In [ ]:
metrics_names = ['TP', 'TN', 'FP', 'FN', 'IS', 'Precision', 'Recall', 'F1']
nb_days = 7

results = np.empty((len(metrics_names), nb_days), dtype=object)
results[:, 0] = metrics_names

print(results)

In [ ]:
for day in range(1, t2p_match_mat.shape[1]): 

    
    manual_matches_elements = [(sublist[0], sublist[day]) if sublist is not None and len(sublist) > day else (None, None) for sublist in table]
    all_cell_reg_elements = [(sublist[0], sublist[day]) if sublist is not None and len(sublist) > day else (None, None) for sublist in all_s2p_idxs_cellreg]

    print("manual_matches_elements:", manual_matches_elements)
    print("all_s2p_idxs_elements:", all_cell_reg_elements)

    metrics = calculate_metrics_pairs(manual_matches_elements, all_cell_reg_elements)
    results[:, day] = metrics

np.save(os.path.join(cellreg_path, 'metrics_table_pairs.npy'), results)
np.save(os.path.join(cellreg_path , 'metrics_table_pairs.npy'), results)

In [ ]:
results_npy = np.load(os.path.join(cellreg_path, 'metrics_table_pairs.npy'), allow_pickle=True)
print(results_npy)

## For 'pairwise" registration

In [ ]:
metrics_names = ['TP', 'TN', 'FP', 'FN', 'IS', 'Precision', 'Recall', 'F1']
nb_days = 7

results = np.empty((len(metrics_names), nb_days), dtype=object)
results[:, 0] = metrics_names

print(results)

In [ ]:
import os
import numpy as np

path = "/Volumes/data_jm_share/data_proc/jm/jm046"
days = ['9', '10', '11', '12', '13', '14']
columns = []

for i, j in enumerate(days): 
    print(i)
    t2p_path = os.path.join(path, f't2p_P8_{j}')
    indices= function(t2p_path, 'plane0')
    indices=np.load(os.path.join(t2p_path, 'true_indices.npy'), allow_pickle=True)
    print(indices.shape)
    # index rows for cells where the first row is in original_indices
    all_s2p_idxs_pairs = []
    for idx in np.sort(original_indices):
        idx_row = np.where(indices[:,0]==idx)
        #print(idx_row)

        if len(idx_row[0]) == 0:
            s2p_idxs = [None]*len(track_ops.all_ds_path)
            #print('here')

        else:
            s2p_idxs = indices[idx_row]
        all_s2p_idxs_pairs.append(s2p_idxs[0])

    all_s2p_idxs_pairs = np.array(all_s2p_idxs_pairs)

    manual_matches_elements = [(sublist[0], sublist[i+1]) if sublist is not None  else (None, None) for sublist in table]
    all_s2p_idxs_elements = [(sublist[0], sublist[1]) if sublist is not None else (None, None) for sublist in all_s2p_idxs_pairs]
#
    print("manual_matches_elements:", manual_matches_elements)
    print("all_s2p_idxs_elements:", all_s2p_idxs_elements)

    metrics = calculate_metrics_pairs(manual_matches_elements, all_s2p_idxs_elements)
    results[:, i + 1] = metrics

np.save(os.path.join(path , 'metrics_table_pw_registration.npy'), results)


In [ ]:
results_npy = np.load(os.path.join(path, 'metrics_table_pw_registration.npy'), allow_pickle=True)
print(results_npy)

## FIGURES

In [ ]:
import numpy as np
import os 
import matplotlib.pyplot as plt

In [ ]:
# Chemins des fichiers NumPy
base_path = '/Volumes/data_jm_share/data_proc/jm'  # Remplacez par le chemin correct
conditions = ['chan0', 'chan1', 'rigid', 'cellreg']
animals = ['jm038', 'jm039', 'jm046']
symbols = {'jm038': 'x', 'jm039': '^', 'jm046': 'o'}
colors = {'jm038': 'blue', 'jm039': 'red', 'jm046': 'green'}


f1_values = {animal: [] for animal in animals}


for animal in animals:
    for condition in conditions:
        metrics = np.load(os.path.join(base_path, animal, condition,'metrics_t2p_all_days.npy'), allow_pickle=True)
        f1_value = metrics[np.where(metrics[:, 0] == 'F1')[0][0], 1]
        f1_values[animal].append(f1_value)
plt.figure(figsize=(3, 3))
for animal in animals:
    plt.plot(conditions, f1_values[animal], symbols[animal], label=animal, color=colors[animal])


plt.ylabel('F1 Score')
plt.yticks([0, 1])
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.legend()
plt.show()

In [ ]:
# Chemins des fichiers NumPy
base_path = '/Volumes/data_jm_share/data_proc/jm'  # Remplacez par le chemin correct
conditions = ['chan0', 'chan1', 'rigid', 'cellreg']
animals = ['jm038', 'jm039', 'jm046']
symbols = {'jm038': 'x', 'jm039': '^', 'jm046': 'o'}
colors = {'jm038': 'blue', 'jm039': 'red', 'jm046': 'green'}


for condition in conditions:
    plt.figure(figsize=(3, 3))
    plt.title(f'{condition} (all days registration)')
    plt.xlabel('Days')
    plt.ylabel('F1 Score')

    # Parcourir chaque animal
    for animal in animals:
        file_path = os.path.join(base_path, animal, condition, 'metrics_table_pairs.npy')
        metrics = np.load(file_path, allow_pickle=True)

        f1_scores = metrics[7, 1:].astype(float)

        # Tracer les valeurs F1
        days = np.arange(1, len(f1_scores) + 1)
        plt.plot(days, f1_scores, label=animal, marker=symbols[animal], color=colors[animal])

    plt.yticks([0, 1])
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.xticks(days, [ 'P9', 'P10', 'P11', 'P12', 'P13', 'P14'])
    plt.legend()
    plt.show()



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Chemins des fichiers NumPy
base_path = '/Volumes/data_jm_share/data_proc/jm'  # Remplacez par le chemin correct
animals = ['jm038', 'jm039', 'jm046']
symbols = {'jm038': 'x', 'jm039': '^', 'jm046': 'o'}
colors = {'jm038': 'blue', 'jm039': 'red', 'jm046': 'green'}

plt.figure(figsize=(3, 3))
plt.title('chan 1 (PW registration)')
plt.xlabel('Days')
plt.ylabel('F1 Score')

# Parcourir chaque animal
for animal in animals:
    file_path = os.path.join(base_path, animal, 'metrics_table_pw_registration.npy')
    metrics = np.load(file_path, allow_pickle=True)

    # Extraire les valeurs F1
    f1_scores = metrics[7, 1:].astype(float)  # La ligne 7 contient les valeurs F1, à partir de la deuxième colonne

    # Tracer les valeurs F1
    days = np.arange(1, len(f1_scores) + 1)
    plt.plot(days, f1_scores, label=animal, marker=symbols[animal], color=colors[animal])

plt.yticks([0, 1])
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xticks(days, [ 'P9', 'P10', 'P11', 'P12', 'P13', 'P14'])
plt.legend()
plt.show()


In [ ]:
base_path = '/Volumes/data_jm_share/data_proc/jm/jm038' 
days = ['9', '10', '11', '12', '13', '14']

for i, j in enumerate(days): 
    t2p_path = os.path.join(base_path, f't2p_P8_{j}')
    track_ops_dict = np.load(os.path.join(t2p_path, 'track2p', 'track_ops.npy'), allow_pickle=True).item()
    track_ops = SimpleNamespace(**track_ops_dict) # create dummy object from the track_ops dictionary
    print(track_ops.reg_chan)